In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


In [2]:
title = "stories"
alpha = 0.3
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]
folders
['stories',
 'stories/FARNON',
 'stories/SRE']

['stories', 'stories/FARNON', 'stories/SRE']

In [3]:
dataset = []

c = False

for i in folders:
    file = open(i+"/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if c == False:
        file_name = file_name[2:]
        c = True
        
    print(len(file_name), len(file_title))

    for j in range(len(file_name)):
        dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))

452 452
0 0
15 15


In [7]:
N = len (dataset)
len(dataset)

467

In [19]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

In [6]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!#$%&()*+-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text
 
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data


<>:14: SyntaxWarning: invalid escape sequence '\]'
<>:14: SyntaxWarning: invalid escape sequence '\]'
C:\Users\aliot\AppData\Local\Temp\ipykernel_22064\728318708.py:14: SyntaxWarning: invalid escape sequence '\]'
  symbols = "!#$%&()*+-./:;<=>?@[\]^_`{|}~"


In [8]:
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

In [9]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])
   

total_vocab_size = len(DF)
print(total_vocab_size)
total_vocab = [x for x in DF]
print(total_vocab[:20])
print(DF['one'])


32350
['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']
444


In [10]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [17]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf


In [12]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [13]:
for i in tf_idf:
    tf_idf[i] *= alpha
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]
len(tf_idf)

34742

In [14]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)

In [15]:
matching_score(10, "a lion dont cry")

Matching Score

Query: a lion dont cry

['lion', 'dont', 'cri']

[266, 288, 267, 265, 87, 0]


In [20]:
print_doc(266)

('c:\\Users\\aliot\\Desktop\\university\\data_retrieval\\practice\\practice_2/stories/lionmosq.txt', 'The Fable of the Lion and the Mosquito')
THE LION AND THE MOSQUITO

   Once upon a time . . . a tiny mosquito started to buzz round a lion he met.
"Go away!" grumbled the sleepy lion, smacking his own cheek in an attempt to 
drive the insect away.
   "Why should I?" demanded the mosquito. "You're king of the jungle, not of 
the air! I'll fly wherever I want and land wherever I please." And so saying, 
he tickled the lion's ear. In the hope of crushing the insect, the lion boxed 
his own ears, but the mosquito slipped away from the now dazed lion.
   "I don't feel it any more. Either it's squashed or it's gone away." But at 
that very moment, the irritating buzz began again, and the mosquito flew into 
the lion's nose. Wild with rage, the lion leapt to his hind legs and started 
to rain punches on his own nose. But the insect, safe inside, refused to 
budge. With a swollen nose and wate

In [21]:
from collections import Counter

def bm25_score(doc, token, counter, doc_len, N, alpha, k):
    tf = counter[token] / doc_len
    df = doc_freq(token)
    
    idf = np.log((N - df + 0.5) / (df + 0.5))
    bm25 = idf * (tf * (k + 1)) / (tf + k)
    
    return bm25 * alpha

def matching_score_bm25(query, N, processed_title, processed_text, alpha=1.0, k=1.5):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("BM25 Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for doc in range(N):
        counter = Counter(processed_title[doc] + processed_text[doc])
        doc_len = len(processed_title[doc] + processed_text[doc])

        for token in np.unique(processed_title[doc]):
            if token in tokens:
                score = bm25_score(doc, token, counter, doc_len, N, alpha, k)
                query_weights[doc] = query_weights.get(doc, 0) + score
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    top_documents = [doc_id for doc_id, _ in query_weights[:10]]
    
    print(top_documents)

matching_score_bm25("a lion dont cry", N, processed_title, processed_text, alpha=1.0, k=1.5)


BM25 Matching Score

Query: a lion dont cry

['lion', 'dont', 'cri']

[266, 288, 267, 265, 87]


In [27]:
%pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from rank_bm25 import BM25Okapi

In [29]:
bm25 = BM25Okapi(processed_text)

def bm25_matching_score_lib(query):
    preprocessed_query = preprocess (query)
    tokens = word_tokenize(str(preprocessed_query))

    print("BM25 Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    scores = bm25.get_scores(tokens)

    relevant_documents = sorted(range(len(scores)), key=lambda i: scores[i],
    reverse=True) [:10]

    print("\n")
    print(relevant_documents)

In [30]:
matching_score(10, "lions tiger")
bm25_matching_score_lib("lions tiger")

Matching Score

Query: lions tiger

['lion', 'tiger']

[0]
BM25 Matching Score

Query: lions tiger

['lion', 'tiger']


[363, 257, 26, 266, 27, 288, 267, 150, 434, 265]


In [24]:
print_doc(266)

('c:\\Users\\aliot\\Desktop\\university\\data_retrieval\\practice\\practice_2/stories/lionmosq.txt', 'The Fable of the Lion and the Mosquito')
THE LION AND THE MOSQUITO

   Once upon a time . . . a tiny mosquito started to buzz round a lion he met.
"Go away!" grumbled the sleepy lion, smacking his own cheek in an attempt to 
drive the insect away.
   "Why should I?" demanded the mosquito. "You're king of the jungle, not of 
the air! I'll fly wherever I want and land wherever I please." And so saying, 
he tickled the lion's ear. In the hope of crushing the insect, the lion boxed 
his own ears, but the mosquito slipped away from the now dazed lion.
   "I don't feel it any more. Either it's squashed or it's gone away." But at 
that very moment, the irritating buzz began again, and the mosquito flew into 
the lion's nose. Wild with rage, the lion leapt to his hind legs and started 
to rain punches on his own nose. But the insect, safe inside, refused to 
budge. With a swollen nose and wate

In [23]:
print_doc(363)

('c:\\Users\\aliot\\Desktop\\university\\data_retrieval\\practice\\practice_2/stories/redragon.txt', 'The Fable of the Red Dragon')
THE RED DRAGON

   Once there was a time, thousands of years ago, when animals were not the 
same as they are now. Except for a few like the lion, the tiger and the 
butterflies, they all looked alike. All were more or less the same height, 
everyone had four legs and it wasn't easy to tell which was which, even though
the elephant did weigh more than the hyena, and the hippo more than the 
gazelle. One day, while all the animals were relaxing in a field, along came a
red dragon, out of breath, crying,
   "We're in danger, folks! The world is about to come to an end!"
   "How do you know?" everyone asked. The dragon replied,
   "I read it in the stars. We must escape!"
   "But where can we go?" they asked him.
   "To another world," he replied. "I'll take you there. I can fly and I'll 
take you to a planet that is safer than this one." Frightened, as they 